In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [36]:
from sift.corpora import wikipedia
from sift.models import text, links

In [5]:
base_path = '/data/wikipedia/20151002/'

In [6]:
corpus = wikipedia.WikipediaCorpus()(sc, base_path + 'dump')

In [9]:
redirects = wikipedia.WikipediaRedirects()(corpus)
docs = wikipedia.WikipediaArticles()(corpus, redirects)

In [27]:
docs.take(1)

[{'_id': 'en.wikipedia.org/wiki/Causa_y_Efecto',
  'links': [{'start': 986,
    'stop': 999,
    'target': u'en.wikipedia.org/wiki/New_York_City'},
   {'start': 93, 'stop': 105, 'target': u'en.wikipedia.org/wiki/Studio_album'},
   {'start': 971, 'stop': 982, 'target': u'en.wikipedia.org/wiki/Gotham_Hall'},
   {'start': 2178,
    'stop': 2192,
    'target': u'en.wikipedia.org/wiki/Miami,_Florida'},
   {'start': 1791,
    'stop': 1808,
    'target': u'en.wikipedia.org/wiki/Latin_Pop_Airplay'},
   {'start': 2702,
    'stop': 2719,
    'target': u'en.wikipedia.org/wiki/Latin_Pop_Airplay'},
   {'start': 465,
    'stop': 484,
    'target': u'en.wikipedia.org/wiki/Ni_Una_Sola_Palabra'},
   {'start': 2122, 'stop': 2129, 'target': u'en.wikipedia.org/wiki/Austria'},
   {'start': 2740,
    'stop': 2760,
    'target': u'en.wikipedia.org/wiki/Latin_Rhythm_Airplay'},
   {'start': 106,
    'stop': 119,
    'target': u'en.wikipedia.org/wiki/Gran_City_Pop'},
   {'start': 2388, 'stop': 2397, 'target': u

In [25]:
vocab = text.TermVocab(max_rank=100000,min_rank=0,lowercase=True,min_df=5)(docs.sample(False, 0.25))

In [26]:
vocab.take(10)

[{'_id': u'the', 'count': 1172125, 'rank': 0},
 {'_id': u'in', 'count': 1135557, 'rank': 1},
 {'_id': u'a', 'count': 1127366, 'rank': 2},
 {'_id': u'of', 'count': 1101586, 'rank': 3},
 {'_id': u'and', 'count': 1008637, 'rank': 4},
 {'_id': u'is', 'count': 997703, 'rank': 5},
 {'_id': u'references', 'count': 958549, 'rank': 6},
 {'_id': u'to', 'count': 889253, 'rank': 7},
 {'_id': u'was', 'count': 804122, 'rank': 8},
 {'_id': u'for', 'count': 725355, 'rank': 9}]

In [34]:
text.EntityMentions(sentence_window=1,lowercase=True)(docs).take(1)

[{'_id': u'en.wikipedia.org/wiki/New_York_City',
  'source': 'en.wikipedia.org/wiki/Causa_y_Efecto',
  'span': (73, 86),
  'text': u'rubio also performed the song in a private concert at the gotham hall in new york city on may 11 promoting gran city pop.'}]

In [31]:
mentions = text.IndexMappedMentions(sentence_window=1,lowercase=True)(sc, docs, vocab)

In [32]:
mentions.take(1)

[{'_id': u'en.wikipedia.org/wiki/New_York_City',
  'source': 'en.wikipedia.org/wiki/Causa_y_Efecto',
  'span': (14, 17),
  'text': [25961,
   18,
   686,
   0,
   447,
   1,
   2,
   574,
   2057,
   13,
   0,
   21394,
   518,
   1,
   35,
   227,
   98,
   11,
   46,
   205,
   3585,
   9860,
   98,
   1770]}]

In [42]:
links.EntityCounts()(docs).take(10)

[{'_id': u'', 'count': 4},
 {'_id': u'www.rsssf.com/tabless/slow97.html', 'count': 2},
 {'_id': u'en.wikipedia.org/wiki/Yuba,_Michigan', 'count': 3},
 {'_id': u'en.wikipedia.org/wiki/Walnut_River_(Kansas)', 'count': 12},
 {'_id': u'www.ctheritage.org/encyclopedia/ct1865_1929/admin_baldwin.htm',
  'count': 2},
 {'_id': u'en.wikipedia.org/wiki/Falling_factorial', 'count': 28},
 {'_id': u'en.wikipedia.org/wiki/WOW_Worship:_Blue', 'count': 5},
 {'_id': u'en.wikipedia.org/wiki/Ekhane_Pinjar', 'count': 2},
 {'_id': u'en.wikipedia.org/wiki/Conditional_execution', 'count': 2},
 {'_id': u'en.wikipedia.org/wiki/Paralititan', 'count': 27}]

In [49]:
links\
    .EntityNameCounts()(docs)\
    .filter(lambda r: len(r['counts']) >= 3)\
    .take(5)

[{'_id': u'XMT',
  'counts': {u'en.wikipedia.org/wiki/Cray_XMT': 1,
   u'en.wikipedia.org/wiki/Extensible_MPEG-4_Textual_Format': 1,
   u'en.wikipedia.org/wiki/XMT': 1},
  'total': 3},
 {'_id': u'<a href="en.wikipedia.org/wiki/New York">New York',
  'counts': {u'en.wikipedia.org/wiki/New_York_(disambiguation)': 1,
   u'en.wikipedia.org/wiki/New_York_Disability_Benefits_Law': 1,
   u'en.wikipedia.org/wiki/New_York_State_Armory_(Poughkeepsie)': 1,
   u'en.wikipedia.org/wiki/Outline_of_New_York': 1,
   u'en.wikipedia.org/wiki/Vehicle_registration_plates_of_New_York': 1},
  'total': 5},
 {'_id': u'Albert Lewis',
  'counts': {u'en.wikipedia.org/wiki/Albert_Gerald_Lewis': 1,
   u'en.wikipedia.org/wiki/Albert_Lewis': 1,
   u'en.wikipedia.org/wiki/Albert_Lewis_(American_football)': 27,
   u'en.wikipedia.org/wiki/Albert_Lewis_(Sheffield_United)': 1,
   u'en.wikipedia.org/wiki/Albert_Lewis_(footballer)': 3,
   u'en.wikipedia.org/wiki/Albert_Lewis_(priest)': 2,
   u'en.wikipedia.org/wiki/Albert_L